In [42]:
import pandas as pd
import seaborn as sns
import numpy as np
import mygene
mg = mygene.MyGeneInfo()

#import sys
#!conda install --yes --prefix {sys.prefix} seaborn
#!pip install mygene

In [ ]:
#output file from regular expressions script, in this case 3UTR (same script was used for 5'UTR and CDS data)

df_MOTIVOS = pd.read_csv('../ExpresionesRegulares_TargetScanCANONICAL_3UTR.csv', sep='\t',\
                index_col=0, names=['gene', 'seq', 'mir', 'motivo', 'apariciones_motivo','zonaMedia', 'LargoZM'])



In [ ]:
df_MOTIVOS.shape

In [ ]:
#Aca el dropduplicates estaba solo genname y mir por eso no me aparecian muchos de los datos de los duplicados

def separarunaCol(df, colname):
    
    
    df['gen_name'] = list(df.reset_index()[colname].str.split('|', expand = True)[1])
    
    return df.sort_values('apariciones_motivo', ascending=False).drop_duplicates(['gen_name', 'mir','motivo'], keep = 'first')
    
    


In [ ]:

df_MOTIVOS = separarunaCol(df_MOTIVOS, 'gene')

In [ ]:
unionConcat = df_MOTIVOS.copy()

In [ ]:
unionConcat.shape

### agregar correlaciones

In [ ]:
cr = pd.read_csv('correlaciones_caluladasconPandas_2020_2p0_POST_BARTEL_Filtrado80porciento_de_los_datos_143_microRNAs.csv', sep='\t',\
                index_col=0)
corr = pd.DataFrame(cr)

#corr.set_index('index', inplace=True)
corr = corr.T
corr.reset_index( inplace=True)
corr['gene_name'] = corr['index'].str.split('|', expand = True)[0]
corr.set_index('index', inplace=True)
corr.set_index('gene_name', inplace=True)

_rnaseq = pd.read_csv('../../PRAD.rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data.data.txt', 
                         sep='\t',
                         index_col= 0)

rnaseq0 = pd.DataFrame(_rnaseq)
rnaseq0 = rnaseq0.replace(0, np.nan)
rnaseq0.dropna(thresh = 50, inplace=True) # me saco muchos outliers mal calculados

rnaseq = rnaseq0.mean(axis=1).to_frame(name='media_exp_gen_RNAseq')
rnaseq.reset_index(inplace=True)
rnaseq[['gen_name', 'id']] = rnaseq['Hybridization REF'].str.split('|', expand=True)
rnaseq.set_index('gen_name', inplace=True)
rnaseq = rnaseq['media_exp_gen_RNAseq'].to_frame()

In [ ]:
corr.drop(index='?',inplace=True)

print(corr.isna().sum().sum())

corr.replace(to_replace=1, value= np.nan, inplace=True)
corr.replace(to_replace=-1, value= np.nan, inplace=True)

print(corr.isna().sum().sum())

In [ ]:
df_zscore = (corr - corr.mean())/corr.std()

In [ ]:
corr = corr.join(df_zscore,  lsuffix = '_spearman', rsuffix='_zscore', how='inner')

#corr = corr.join(rnaseq, how= 'inner')

In [ ]:
#unionConcat.set_index('gen_name', inplace=True)

In [ ]:
corr = corr.stack().reset_index()
corr[['miroRNA_name', 'MIMAT']] = corr['level_1'].str.split('|', expand=True)

corr.columns = ['gen_name', 'miRyID','CorrOrZscore', 'miroRNA_name', 'MIMAT']
#corr

In [ ]:
#Elimino los .2 y le saco el .1 a los otros porque no tengo esa distincion en los datos de corr
unionConcat = unionConcat[~unionConcat['mir'].str.contains('\.2')]#.sort_values('mir')['mir'].unique()
unionConcat['mir'] = unionConcat['mir'].str.replace('\.1','')

#### Union de los archivos

In [ ]:
merged = unionConcat.merge(corr, how='left',left_on=['mir', 'gen_name'], right_on=['miroRNA_name', 'gen_name'], indicator=True) 

In [ ]:
unionConcat.shape

In [ ]:
merged.shape

In [ ]:
#merged.to_csv('motivos_regexp_unido_con_corr_zscore_meanExpr.csv', sep='\t')


In [ ]:
probando_recuperar = merged.sort_values('CorrOrZscore', ascending=False).drop_duplicates('gen_name', keep='first')

faltan = set(probando_recuperar[probando_recuperar['_merge'] == 'left_only']['gen_name'])

index_conteos = mg.querymany(faltan, scopes='symbol', species=9606, as_dataframe = True, fields = 'entrezgene', returnall =False)

index_conteos.reset_index(inplace=True)
index_conteos.drop_duplicates('query', keep='first', inplace=True)
index_conteos.set_index('entrezgene', inplace=True)

cr = pd.read_csv('correlaciones_caluladasconPandas_2020_2p0_POST_BARTEL_Filtrado80porciento_de_los_datos_143_microRNAs.csv', sep='\t',\
                index_col=0)
corr2 = pd.DataFrame(cr)
#corr2.set_index('index', inplace=True)
corr2 = corr2.T
corr2.reset_index( inplace=True)
corr2[['gene_name', 'Gen_ID']] = corr2['index'].str.split('|', expand = True)
corr2.set_index('index', inplace=True)
corr2.set_index('gene_name', inplace=True)
corr2.replace(to_replace=1, value= np.nan, inplace=True)
corr2.replace(to_replace=-1, value= np.nan, inplace=True)

corr2 = corr2.join(df_zscore,  lsuffix = '_spearman', rsuffix='_zscore', how='inner')

#corr2 = corr2.join(rnaseq, how= 'inner')
corr2.set_index('Gen_ID', inplace=True)

recuperados = index_conteos['query'].to_frame().join(corr2, how= 'inner')
recuperados.set_index('query', inplace=True)

print('se recueran',recuperados.shape[0] )
#prueba_recupero = merged.combine_first(recuperados)

In [ ]:

recuperadosunstack = recuperados.stack().reset_index()
recuperadosunstack[['miroRNA_name', 'MIMAT']] = recuperadosunstack['level_1'].str.split('|', expand=True)
recuperadosunstack.columns = ['gen_name', 'miRyID','CorrOrZscore', 'miroRNA_name', 'MIMAT']

#prueba_recupero = merged.combine_first(recuperadosunstack)

In [ ]:
merged['gen_name'] = merged['gen_name'].astype(str)
merged['miRyID'] = merged['miRyID'].astype(str)

recuperadosunstack['gen_name'] = recuperadosunstack['gen_name'].astype(str)
recuperadosunstack['miRyID'] = recuperadosunstack['miRyID'].astype(str)

merged.set_index(['gen_name', 'miRyID'], inplace=True)
recuperadosunstack.set_index(['gen_name', 'miRyID'], inplace=True)

prueba_recupero = merged.combine_first(recuperadosunstack)
prueba_recupero.reset_index(inplace=True)

In [ ]:
merged.reset_index(inplace=True)
recuperadosunstack.reset_index(inplace=True)

In [ ]:
print(prueba_recupero['miroRNA_name'].nunique())
print(prueba_recupero['mir'].nunique())

In [ ]:
#Le saco lo que no tiene valor de Zscore o de correlacion
prueba_recupero.dropna(subset=['CorrOrZscore'], inplace=True)

In [ ]:
print('Numero de genes que quedan despues de unir los dfs',len(set(merged[(merged['_merge'] == 'both') & (~merged['CorrOrZscore'].isna())]['gen_name'])))

In [ ]:
print('Numero de genes que quedan despues de unir los dfs', len(set(prueba_recupero[~prueba_recupero['CorrOrZscore'].isna()]['gen_name'])))

In [ ]:
print('Numero de genes que quedan ANTES de unir los dfs',len(set(unionConcat.index)))

In [ ]:
#merged[merged['miroRNA_name'].isna()]#['mir'].unique()
merged['miroRNA_name'].nunique()

In [ ]:
print(prueba_recupero['miroRNA_name'].nunique())
print(prueba_recupero['mir'].nunique())

In [ ]:
print(merged.shape)
print(prueba_recupero.dropna(subset=['mir']).shape)

In [ ]:
prueba_recupero.dropna(subset=['mir'])['mir'].nunique()

In [ ]:
#prueba_recupero.to_csv('motivos_regexp_s6_s7_s6m41316_largoZM_unido_con_corr_zscore_meanExpr_TODOS_conservEvol.csv', sep='\t')

In [ ]:
#prueba_recupero.drop(columns = ['seq']).to_csv('motivos_TARGETSCAN_conservEVOL_sinsequencia_3UTR_correlaciones_filtrado80porciento.csv', sep='\t')

In [ ]:
print('Termino el script')

In [ ]:
#prueba_recupero[(prueba_recupero['miroRNA_name'] == 'hsa-let-7a-5p') & (prueba_recupero['gen_name'].duplicated(keep=False)) & (prueba_recupero['MIMAT'].str.contains('_zscore'))].sort_values(['gen_name', 'miroRNA_name'])

In [ ]:
df_numero_interacciones = merged[merged.miRyID.str.contains('zscore')]

df_numero_interacciones.apariciones_motivo.sum()